不可能，絕對不可能！

統計一下母體之後丟上去！

分數就有 0.55569 了！

太離譜了！太離譜辣！

In [1]:
import pandas as pd

# index, session_id, song_id, unix_played_at, play_status, login_type, listening_order
test_source = pd.read_parquet("../../datagame-2023/label_test_source.parquet")
# index, song_id, artist_id, song_length, album_id, language_id, album_month
meta_song = pd.read_parquet("../../datagame-2023/meta_song.parquet")

In [2]:
test_session_to_songs = test_source.groupby('session_id')['song_id'].apply(list).to_dict()

In [3]:
from tqdm import tqdm

song_to_artist = dict()
song_to_album = dict()

for row in tqdm(meta_song.itertuples(), total=len(meta_song)):
    if not pd.isna(row.artist_id):
        song_to_artist[row.song_id] = int(row.artist_id)

    if not pd.isna(row.album_id):
        song_to_album[row.song_id] = int(row.album_id)

100%|██████████| 1030712/1030712 [00:03<00:00, 313915.60it/s]


In [4]:
del test_source, meta_song

In [5]:
from collections import Counter


class myConditionalFreqDist:
    def __init__(self):
        self._data = {}
        self.values = [1, 0.63, 0.5, 0.43, 0.38]  # ndcg 的加權分數

    def __getitem__(self, key):
        return self._data.get(key, {})

    def __setitem__(self, key, value):
        self._data[key] = value

    def inc(self, condition, sample, index):
        if condition in self._data:
            if sample in self._data[condition]:
                self._data[condition][sample] += self.values[index]
            else:
                self._data[condition][sample] = self.values[index]
        else:
            self._data[condition] = {sample: self.values[index]}

    def most_common(self, condition, n=None):
        if condition in self._data:
            counter = Counter(self._data[condition])
            return counter.most_common(n)
        else:
            return []

In [6]:
import pickle

In [7]:
with open('cfd_3grams_test', 'rb') as file:
    cfd_3grams = pickle.load(file)

In [8]:
with open('cfd_4grams_test', 'rb') as file:
    cfd_4grams = pickle.load(file)

In [9]:
with open('cfd_5grams_test', 'rb') as file:
    cfd_5grams = pickle.load(file)

In [15]:
from tqdm import tqdm

submit_rows = []
results_index = 0
part_repeated = 0
part4321 = 0
part321 = 0
part21 = 0
part5432 = 0
part432 = 0
part6543 = 0
part543 = 0
part32 = 0
part321new1 = 0
part21new1 = 0
part21new1new2 = 0
part1new1new2 = 0
part_mle = 0
same_song_count = 0

for key in tqdm(test_session_to_songs.keys(), total=len(test_session_to_songs.keys())):
    query_song = test_session_to_songs[key][:]

    # 預測下一首歌曲
    predicted_songs = []

    '''
    用戶行為
    '''
    user_code = 0  # 0=正常用戶，1=單一藝人循環，2=單一專輯循環

    is_same_artist = True
    is_same_album = True
    artist = song_to_artist.get(query_song[-10], '')
    album = song_to_album.get(query_song[-10], '')
    for query in query_song[-10:]:
        if song_to_artist.get(query, ' ') != artist:
            is_same_artist = False
        if song_to_album.get(query, ' ') != album:
            is_same_album = False
    if is_same_artist:
        user_code = 1
    if is_same_album:
        user_code = 2

    if len(set(query_song[-2:])) == 1:  # 如果倒數兩首都是重複同一首歌
        predicted_songs.append(query_song[-1])
        part_repeated += 1

    is_same_song = False
    if len(set(query_song)) < 16:
        is_same_song = True  # 可能會聽前面的歌的用戶

    '''
    5 4 3 grams 依次過濾，分數主要來源來自這三個，其他其實都只是加分項
    '''

    if len(predicted_songs) < 5:
        start_song = (query_song[-4] + query_song[-3] + query_song[-2] + query_song[-1])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs:
                part4321 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-3] + query_song[-2] + query_song[-1])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs:
                part321 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-2] + query_song[-1])
        next_songs = cfd_3grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part21 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    '''
    賭狗行為沒賭到，開始找如果不選 query_song[-1] 的選項
    依序是倒數
    5432
    432
    6543
    543
    32 的預測架構，盡量找長序列，然後加43雖然能找得到東西，但太遠了相關性就掉得厲害
    可能可以再改改順序，但算了不凹分
    '''
    if len(predicted_songs) < 5:
        start_song = (query_song[-5] + query_song[-4] + query_song[-3] + query_song[-2])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part5432 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-4] + query_song[-3] + query_song[-2])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part432 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-6] + query_song[-5] + query_song[-4] + query_song[-3])
        next_songs = cfd_5grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part6543 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-5] + query_song[-4] + query_song[-3])
        next_songs = cfd_4grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part543 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    if len(predicted_songs) < 5:
        start_song = (query_song[-3] + query_song[-2])
        next_songs = cfd_3grams.most_common(start_song)

        index = 0
        while index < len(next_songs) and len(predicted_songs) < 5:
            next_song, times = next_songs[index]
            index += 1
            if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                part32 += 1
                predicted_songs.append(next_song)
            if len(predicted_songs) >= 5:
                break

    '''
    賭狗上線
    預測順序是 5gram 4gram 先加入第一個變數來預測
    接下來是 5gram 4gram 加入第二個變數來預測，因為加入越後面的變數越容易偏掉（可能冷門歌曲亂跳)
    加入第三個之後模型預測效能會下降 (0.56056->0.56051)
    '''
    if 0 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            start_song = (query_song[-3] + query_song[-2] + query_song[-1] + predicted_songs[i])
            next_songs = cfd_5grams.most_common(start_song)

            index = 0
            while index < len(next_songs) and len(predicted_songs) < 5:
                next_song, times = next_songs[index]
                index += 1
                if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                    part321new1 += 1
                    predicted_songs.append(next_song)
                if len(predicted_songs) >= 5:
                    break

            if len(predicted_songs) >= 5:
                break

    if 0 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            start_song = (query_song[-2] + query_song[-1] + predicted_songs[i])
            next_songs = cfd_4grams.most_common(start_song)

            index = 0
            while index < len(next_songs) and len(predicted_songs) < 5:
                next_song, times = next_songs[index]
                index += 1
                if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                    part21new1 += 1
                    predicted_songs.append(next_song)
                if len(predicted_songs) >= 5:
                    break

            if len(predicted_songs) >= 5:
                break

    if 1 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            for j in range(len(predicted_songs)):
                if i == j:
                    continue
                start_song = (query_song[-2] + query_song[-1] + predicted_songs[i] + predicted_songs[j])
                next_songs = cfd_5grams.most_common(start_song)

                index = 0
                while index < len(next_songs) and len(predicted_songs) < 5:
                    next_song, times = next_songs[index]
                    index += 1

                    if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                        part21new1new2 += 1
                        predicted_songs.append(next_song)
                    if len(predicted_songs) >= 5:
                        break

                if len(predicted_songs) >= 5:
                    break

    if 1 < len(predicted_songs) < 5:
        for i in range(len(predicted_songs)):
            for j in range(len(predicted_songs)):
                if i == j:
                    continue
                start_song = (query_song[-1] + predicted_songs[i] + predicted_songs[j])
                next_songs = cfd_4grams.most_common(start_song)

                index = 0
                while index < len(next_songs) and len(predicted_songs) < 5:
                    next_song, times = next_songs[index]
                    index += 1

                    if next_song not in predicted_songs and (next_songs not in query_song or is_same_song):
                        part1new1new2 += 1
                        predicted_songs.append(next_song)
                    if len(predicted_songs) >= 5:
                        break

                if len(predicted_songs) >= 5:
                    break

    '''
    pyserini 區域
    預計做 MLE and BM25 的混合模型
    '''
    for i in range(1, 21):
        if len(predicted_songs) >= 5:
            break
        if query_song[-i] not in predicted_songs:
            predicted_songs.append(query_song[-i])
            same_song_count += 1

    if len(predicted_songs) < 5:
        predicted_songs.append('18a62aea3e0e67e21ea56c125c29c474')

    if len(predicted_songs) < 5:
        predicted_songs.append('85422f927d88358292985cb319d216fa')

    if len(predicted_songs) < 5:
        predicted_songs.append('ef71212934e35b400232a0cd8e7e67a2')

    if len(predicted_songs) < 5:
        predicted_songs.append('4a27803802d6090389507671d8aed6eb')

    if len(predicted_songs) < 5:
        predicted_songs.append('bc58d186329eda8b23e510ba98b2c1ea')

    # print("Predicted Songs:", predicted_songs)
    submit_rows.append([key, *predicted_songs])

100%|██████████| 143064/143064 [00:47<00:00, 3027.74it/s]


In [16]:
# 重複的歌曲
print(part_repeated)
# 第一層（正統 n gram)
print(part4321, part321, part21)
# 第二層（擴展 n gram 範圍）
print(part5432, part432, part6543, part543, part32)
# 第三層（拿預測預測）
print(part321new1, part21new1, part21new1new2, part1new1new2, part_mle)
print(same_song_count)

8327
144615 26481 126802
86744 3417 80830 2351 32772
36 64 24 694 0
198779


In [14]:
columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
output_df = pd.DataFrame(submit_rows, columns=columns)
output_df.to_csv('only_ngrams.csv', index=False)

In [ ]:
output_df

In [ ]:
'''
150299 27066 128429
87175 3417 80830 2351 32772
3 6 1 633 202159
'''

In [ ]:
output_df.isnull().sum()